## Overall Concept

The decision variables $P_{i,j}$, the amount of practice of task $i$ during
time slot $j$ (e.g., a day in a sequence of days), are organized into a tableau like the following:

|          | $1$   | $2$ |$\ldots$|$m$|
|:--------:|:-------:|:-------:|:--------:|:--------:|
|$item_1$  | $P_{1,1}$    | $P_{1,2}$   |   |   $P_{1,m}$      |
|$item_2$  | $P_{2,1}$    | $P_{2,2}$   |  |    $P_{2,1}$     |
|$\vdots$  |          |   |    $\ddots$     |       |
|$item_{n}$| $P_{n,1}$|$P_{n,2}$   |        | $P_{n,m}$       |

where $n =NITEMS$, and $m = NSLOTS$.

Rows are the items to be studied and columns are time 
slots (e.g., days). Constraints are expressed in terms of sums over
rows and columns, or in the 
case of the density constraint, over a series of adjacent columns.

## LP Formulation

### Counts and Indices

\begin{align*}
&NITEMS             && \text{Number of items to schedule} \\
&NSLOTS             && \text{Number of time slots available} \\
&i \in \{0, 1,  \ldots, NITEMS-1 \} && \text{Index over items}\\
&t \in \{0, 1, \ldots, NSLOTS-1 \} && \text{Index over time slots} \\
&t_\delta \in \{0, 1, \ldots, WINSZ-1\} && \text{Index over time window}
\end{align*}

See below for the use of $t_\delta$ and $WINSZ$ to express 
limits on practice "density".

### Data

\begin{align*}
&ERGUSED_i          && \text{Amount of mental "energy" required per unit of practice time for task i ("ergs" of energy)} \\
&ERGEVAIL_t        && \text{Amount of mental "energy" available per day ("ergs"/day)}\\
&TIMEAVAIL_t       && \text{Amount of time resource available during a training slot $t$ (time units per day)} \\
&TIMEPER_i         && \text{Practice time needed to master item $i$} \\
&MINPERWIN_i  && \text{Minimum number of training slots for item $i$ within a window (days)} \\
&MAXPERWIN_i  && \text{Maximum number of training slots for item $i$ within a window (days)} \\
&WINSZ            && \text{Used to express minimum and maximum "density" of practice days (days)}\\
\end{align*}

### Variables

\begin{align*}
&P_{i,t}      && \text{Amount of practice in time slot $t$ for item $i$ (number of units of time study)}\\
&ERG_{t}           && \text{Energy expended on a given day (macro over $P$)}\\
&ERGDEC_t     && \text{Amount by which energy expenditure decreased at $t$ from $t-1$}\\
&ERGINC_t     && \text{Amount by which energy expenditure increased at $t$ from $t-1$}\\
&TIMESOFAR_{i,t}      && \text{Amount of practice for item $i$ up to and including time $t$ (Macro over $P$)}\\
\end{align*}

where $P_{i,t} \in \mathbb Z_{\ge 0};\ ERG_{t}, ERGINC_{t}, ERGDEC_{t}\in \mathbb R_{\ne 0}$.


### Objective and Constraints

Minimize $\underbrace{\sum_{i, t} TIMEPER_i P_{i,t}}_{\text{(O1) Overall time spent}} 
        + \underbrace{\sum_{t=1}{(ERGINC_t - ERGDEC_t)}}_{\text{(O2) Energy smoothing}}
        + \underbrace{\sum_{i,t}TIMESOFAR_{i,t}}_{\text{(O3) Calendar time}}$

s.t.
\begin{align*}
&& \sum_{i}{P_{i,t}} &\leq TIMEAVAIL_t\ \forall\ t 
  && \text{(C1) No more time allocated to time slot than available} \\
&& \sum_{t}{P_{i,t}} &\geq TIMEPER_i\   \forall\ i 
  && \text{(C2) Practice time greater than required to learn item} \\
&& \sum_{i}{P_{i,t + t_\delta}}  &\geq MINPERWIN\ \forall i, t \forall t_\delta 
  && \text{(C3) Practice spacing - min. amount in window} \\
&& \sum_{i}{P_{i,t + t_\delta}}  &\leq MAXPERWIN\ \forall i, t
  && \text{(C4) Practice spacing for rest - max. amount in window}\\
&& TIMESOFAR_{i,t} &= \sum_i{\sum_{t0=0 \ldots t}{P_{i,t0}}}\ \forall i,t 
  && \text{(C5) Defn. Used in objective to bias toward finishing earlier}\\
&& ERG_{t} &= \sum_i{ERGUSED_i \times P_{i,t}}\ \forall t
  && \text{(C6) Defn. Used to define increase/decrease for smoothing.}\\
&& ERGINC_{t+1} - ERGDEC_{t+1} &= ERG_{t+1} - ERG_{t}\ \forall t
  && \text{(C7) Defn. Change in either direction. One always zero.}\\
\end{align*}


### Discussion

The objective function is composed of three positive terms
to be minimized. The first term (O1)
captures the amount the sum of the total amount of practice time across all
items. The second term (O2) is the relative energy expenditure on practice
from day to day. This smoothes energy expenditure, which has the effect
of preferring spreading around the more energy-sapping work, and increasing
the diversity with respect to intensity on any given day. Finally
(O3) was the only way I could figure out how to express a preference
for completing everything as early as possible.

The first two constraints are essentially supply and demand constraints. 
The time spent practicing cannot exceed the time available for practice (C1)
and each item requires that a minimum amount of practice is done to master the
item (C2).

The constraints (C3) and (C4) are contraints that limit the number of consecutive
days spent practicing. They are expressed in terms of a time window
parameter. The goal is to ensure an adequate number of days spent resting while
ensuring that practice is resumed before too long passes.

The constraints (C5), (C6) and C(7) relate to energy expenditure. The equality
constraints define the difference in energy expenditure from day to day. The
difference $ERGINC_{t+1} - ERGDEC_{t+1}$ of two positive variables captures
this change. Either $ERGINC_{t+1}$ or $ERGDEC_{t+1}$ will always be driven
to zero by the minimization.

Finally, with (C8) we require that the amount of practice of a given item on a given
day $P_{i,t}$ is a non-negative *integer* to enforce the fixed-size
practice sessions, and that all other variables are non-negative 
real numbers.

There are several known issues with this formulation that I will need to address:

* Incommensurability of the three terms in the objective function. A common metric that captures utility with appropriate scaling factors is needed. This should correlate
with the expected "satisfaction" experience by the practicer. I'll need to add
coefficients to the required Data that encode this.

* It's not clear that $TIMESOFAR_{i,t}$ is needed in the model (O3). It's not clear that preference should be given to completing tasks as early as possible. That is, it's
not clear a priori that given excess days that "packing" the schedule is best. It may
be that this is needs to be scaled in in some way (see above).

* There are no constraints related to energy except for the smoothing constraint. 
It is probably desirable to specify minimum and maximum expenditures in some way.

### Diagnostic Metrics

The slacks and excesses are useful information for a user of the scheduling tool
if presented properly. In addition, some efficiency-related metrics are
probably going to be useful:

* Re-practice ratio for an item $i$. Amount of practice beyond required beyond $TIMEPER_i$ over
required practice (i.e., excess for the constraint). This is:
  
$$
R_i = \frac{\sum_t{P_{i,t}} - TIMEPER_i}{TIMEPER_i}
$$

* The most and least efficient tasks are then $arg\,min_i\ r$ and $arg\,imax_i\ r$

* Daily intensity is:

$$
I_t = \frac{ERG_{t}}{TIMEAVAIL_t}
$$

The sums of these over all items may also be useful to present to the user.